In [1]:
import sys
sys.path.insert(1, '../src/')
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.nn.utils import vector_to_parameters, parameters_to_vector
import matplotlib.pyplot as plt
import time
import os
import copy
import utils
from torchsummary import summary
import models
import random
import torch.nn.functional as F
import shutil
from torchsummary import summary
import torch
import torch.nn.functional as F
from PIL import Image
import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import torchvision
from torchvision import transforms
from captum.attr import LayerConductance
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
from copy import deepcopy
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
import torch.utils.data
import math

In [2]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [3]:
class args:
    model_file='iid_clean_model.pt'
    data='fmnist'
    bs = 256
    device = 'cuda:0'
    lr = 0.1
    moment = 0.9
    wd = 0
    epochs=50
    nesterov = True
    base_class = 5
    target_class = 7
    poison_frac = 0.05
    pattern_type='+'
    top_frac = 1e-2

In [4]:
train_dataset, val_dataset = utils.get_datasets(args.data)
idxs = (val_dataset.targets == args.base_class).nonzero().flatten().tolist()
poisoned_val_set = utils.DatasetSplit(copy.deepcopy(val_dataset), idxs)
utils.poison_dataset(poisoned_val_set.dataset, args, poison_all=True)

In [5]:
val_loader =  DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=2, pin_memory=True)
poisoned_val_loader = DataLoader(poisoned_val_set, batch_size=args.bs, shuffle=False, num_workers=0, pin_memory=True) 

In [6]:
model_clean = torch.load(args.model_file).eval()
model_adv = torch.load('iid_nodefense_model.pt')
criterion = nn.CrossEntropyLoss().to(args.device)
model_params = parameters_to_vector(model_adv.parameters())

In [7]:
def comp_diag_fisher(model_params, data_loader):
    
    model = models.get_model(args.data)
    vector_to_parameters(model_params, model.parameters())
    params = {n: p for n, p in model.named_parameters() if p.requires_grad}
    precision_matrices = {}
    for n, p in deepcopy(params).items():
        p.data.zero_()
        precision_matrices[n] = p.data
        
        
    model.eval()
    for _, (inputs, labels) in enumerate(data_loader):
        model.zero_grad()
        inputs, labels = inputs.to(device=args.device, non_blocking=True),\
                                labels.to(device=args.device, non_blocking=True).view(-1, 1)
        outputs = model(inputs)
        log_all_probs = F.log_softmax(outputs, dim=1)
        target_log_probs = outputs.gather(1, labels)
        batch_target_log_probs = target_log_probs.sum()
        batch_target_log_probs.backward()
        
        for n, p in model.named_parameters():
            precision_matrices[n].data += (p.grad.data ** 2) / len(data_loader.dataset)
            
    return parameters_to_vector(precision_matrices.values()).detach()

In [8]:
adv_params = parameters_to_vector(model_adv.parameters())
clean_params = parameters_to_vector(model_clean.parameters())
_, indices = comp_diag_fisher(adv_params, poisoned_val_loader).sort()

In [9]:
top_indices_idx = math.floor(len(indices)*args.top_frac)

In [10]:
top_params_adv = adv_params[indices[-top_indices_idx:]]

In [11]:
top_params_clean = clean_params[indices[-top_indices_idx:]]

In [12]:
(torch.sign(top_params_clean) == torch.sign(top_params_adv)).sum().item() / len(indices)

0.004903815541861616

In [13]:
indice =  random.sample(range(len(clean_params)), 100)
indice = torch.tensor(indice)


In [14]:
sampled_values = clean_params[indice]

In [15]:
clean_params[indice[0]]

tensor(-0.0027, device='cuda:0', grad_fn=<SelectBackward>)

In [20]:
t = sampled_values.cpu().detach().numpy()

In [22]:
torch.Tensor(np.intersect1d(t, t))

tensor([-5.5050e-01, -1.1130e-01, -9.1826e-02, -8.0111e-02, -6.1961e-02,
        -5.3785e-02, -5.3553e-02, -5.3233e-02, -5.0382e-02, -4.6156e-02,
        -4.6036e-02, -4.6009e-02, -4.5807e-02, -4.5443e-02, -3.8943e-02,
        -3.8536e-02, -3.7964e-02, -3.6003e-02, -3.2627e-02, -2.9844e-02,
        -2.9557e-02, -2.8984e-02, -2.5639e-02, -2.4291e-02, -2.2209e-02,
        -2.1418e-02, -1.9410e-02, -1.9051e-02, -1.8818e-02, -1.7951e-02,
        -1.6195e-02, -1.4247e-02, -1.3428e-02, -1.3398e-02, -1.2479e-02,
        -1.0225e-02, -8.8634e-03, -8.3835e-03, -8.2482e-03, -7.9961e-03,
        -7.9933e-03, -7.3583e-03, -7.0098e-03, -5.9064e-03, -5.0853e-03,
        -4.8861e-03, -4.5651e-03, -3.0822e-03, -2.9881e-03, -2.6612e-03,
        -1.5203e-03, -3.1622e-04,  1.3897e-03,  1.9241e-03,  2.0392e-03,
         2.3581e-03,  2.4028e-03,  2.5994e-03,  3.1765e-03,  4.1285e-03,
         4.9352e-03,  5.0809e-03,  5.5784e-03,  5.6284e-03,  6.2095e-03,
         6.6068e-03,  6.6484e-03,  7.3526e-03,  8.0